In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from collections import namedtuple
#from pyspark.streaming import StreamingContext

## Setup Spark Streaming

In [4]:
spark = SparkSession.builder.appName('twitter_structured_streaming').getOrCreate()
#sc = spark.sparkContext
#ssc = StreamingContext(sc, 10)

In [5]:
#socket_stream = ssc.socketTextStream("0.0.0.0", 5555)

In [17]:
#time_window = 10
#lines = socket_stream.window(time_window)
lines = spark.readStream \
    .format("socket") \
    .option("host", '0.0.0.0') \
    .option("port", 5555) \
    .load()

In [18]:
#fields = ("tag","count")
#Tweet = namedtuple('Tweet',fields)

In [19]:
sep = ',|,lksd\\|.k'

split_cols = split(lines['value'],sep)
hashtags = lines.withColumn('created_at',split_cols.getItem(0)).withColumn('user_name',split_cols.getItem(1)) \
    .withColumn('hashtag',split_cols.getItem(2)).drop('value')

In [20]:
htCounts = hashtags.groupBy("hashtag").count()

In [21]:
lines_c = lines.groupBy('value').count()

In [25]:
query = lines_c.writeStream.queryName('count').outputMode('complete').format('memory').start()

In [24]:
query.stop()

In [29]:
spark.sql("select * from count").toPandas()

,value,count


In [27]:
htCounts

DataFrame[hashtag: string, count: bigint]